# 프로젝트: 한국어 데이터로 챗봇 만들기

- 영어로 만들었던 챗봇을 한국어 데이터로 바꿔서 훈련시키기

## 목차
- 1. 프로젝트 개요
- 2. 데이터 수집하기
- 3. 데이터 전처리하기
- 4. Subword TextEncoder 사용하기
- 5. 모델 구성하기
- 6. 모델 평가하기

## 1. 프로젝트 개요

### 트랜스포머의 인코더와 디코더
- 트랜스포머 또한 번역기와 마찬가지로 기본적으로 인코더와 디코더 구성 가지고 있음
- 입력 문장을 넣으면 출력 문장을 내뱉고 있음
![트랜스포머.png](./images/트랜스포머.png)
- [출처](http://jalammar.github.io/illustrated-transformer/)
- 위의 블랙박스로 가려져 있는 트랜스포머의 내부 구조를 열어보면 아래와 같음
![트랜스포머2.png](./images/트랜스포머2.png)
- [출처](http://jalammar.github.io/illustrated-transformer/)
- 초록색 색깔의 도형을 인코더 층(Encoder layer), 핑크색 색깔의 도형을 디코더(Decoder layer)라고 하였을 때
- 입력 문장은 누적해 쌓아 올린 인코더의 층을 통해서 정보를 뽑아내고, 디코더는 누적해 쌓아올린 디코더의 층을 통해서 출력 문장의 단어를 하나씩 만들어가는 구조 갖고 있음
- 그리고 그 내부를 조금 더 확대해 보면 아래와 같이 톱니바퀴처럼 맞물려 돌아가는 여러 가지 부품들로 구성되어 있음
![트랜스포머3.png](./images/트랜스포머3.png)
- [출처](http://jalammar.github.io/illustrated-transformer/)

### 트랜스포머의 입력 이해하기
![포지셔널인코딩.png](./images/포지셔널인코딩.png)
- 많은 자연어 처리 모델들은 텍스트 문장을 입력으로 받기 위해 단어를 임베딩 벡터로 변환하는 벡터화 과정 거침
- 트랜스포머 또한 그 점에서는 다르 모델들과 다르지 않음
- 하지만 트랜스포머 모델의 입력 데이터 처리에는 RNN 계열의 모델들과 다른 점이 1가지 있음
- 바로 임베팅 벡터에 어떤 값을 더해준 뒤에 입력으로 사용한다는 점
- 그 값은 바로 위 그림에서의 '포지셔널 인코딩(positional Encoding)'에 해당하는 부분
- 위 그림에서 인코더의 입력 부분을 조금 더 확대해 본다면 아래 그림과 같을 것
![포지셔널인코딩2.png](./images/포지셔널인코딩2.png)
- 이렇게 해주는 이유는 트랜스포머는 입력을 받을 때, 문장에 있는 단어들을 1개씩 순차적으로 받는 것이 아니라, 문장에 있는 모든 단어를 한꺼번에 입력으로 받기 때문
- 트랜스포머가 RNN과 결정적으로 다른 점이 바로 이 부분
- RNN에는 어차피 문장을 구성하는 단어들이 어순대로 모델에 입력되므로, 모델에게 따로 어순 정보를 알려줄 필요가 없었음
- 그러나 문장에 있는 모든 단어를 한꺼분에 문장 단위로 입력받는 트랜스포머는 자칫 'I ate lunch'와 'lunch ate I'를 구분할 수 없을지도 모름
- 그래서 같은 단어라도 그 단어가 문장의 몇 번째 어순으로 입력되었는지를 모델에 추가로 알려 주기 위해, 단어의 임베딩 벡터에다가 위치 정보를 가진 벡터(Positional Encoding) 값을 더해서 모델의 입력으로 삼는 것
![포지셔널인코딩3.png](./images/포지셔널인코딩3.png)
- 포지셔널 인코딩의 벡터값은 위의 수식에 의해 정해짐
- 사인 함수와 코사인 함수의 그래프를 상기해보면 요동치는 값의 행태를 생각해 볼 수 있음
- 트랜스포머는 사인 함수와 코사인 함수의 값을 임베딩 벡터에 더해줌으로써 단어의 순서 정보를 더하여 줌
- 위의 두 함수에서는 pos, i, d model등 생소한 변수들이 있음
- 위의 함수를 이해하기 위해서는 위에서 본 임베딩 벡터와 포지셔널 인코딩의 덧셈은 사실 임베딩 벡터가 모여 만들어진 문장 벡터 행렬과 포지셔널 인코딩 행렬의 덧셈 연산을 통해 이루어진다는 점 이해
![포지셔널인코딩4.png](./images/포지셔널인코딩4.png)
- d model은 임베딩 벡터의 차원을 의미하고 있고, pos는 입력 문장에서의 임베딩 벡터의 위치를 나타내며, i는 임베딩 벡터 내의 차원의 인덱스를 의미
- 이렇게 임베딩 행렬과 포지셔널 행렬이라는 두 행렬을 더함으로써 각 단어 벡터에 위치 정보를 더해주게 되는 것

### 진행에 필요한 패키지 import

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_datasets as tfds
import os
import re
import numpy as np
import matplotlib.pyplot as plt
print('슝=3')

슝=3


## 2. 데이터 수집하기

- 한국어 챗봇 데이터는 송영숙님이 공개한 챗봇 데이터 사용
- 아래의 링크에서 다운로드 받을 수 있음
- [songs/Chatbot_data](https://github.com/songys/Chatbot_data/blob/master/ChatbotData.csv)
- Cloud shell에서 아래 명령어 입력
```python
$ mkdir -p ~/aiffel/transformer_chatbot/data/
$ ln -s ~/data/*. ~/aiffel/transformer_chatbot/data/
```

In [2]:
data = pd.read_csv(os.getenv('HOME') + '/aiffel/transformer_chatbot/data/ChatbotData .csv')
print(data.shape)
data.head()

(11823, 3)


,Q,A,label
0,12시 땡!,하루가 또 가네요.,0
1,1지망 학교 떨어졌어,위로해 드립니다.,0
2,3박4일 놀러가고 싶다,여행은 언제나 좋죠.,0
3,3박4일 정도 놀러가고 싶다,여행은 언제나 좋죠.,0
4,PPL 심하네,눈살이 찌푸려지죠.,0


## 3. 데이터 전처리하기

- 영어 데이터와는 전혀 다른 데이터인 만큼 영어 데이터에 사용했던 전처리와 일부 동일한 전처리도 필요하겠지만 전체적으로 다른 전처리 수행해야 할 수도 있음

- 이를 위한 전처리 함수는 다음과 같음
- 이번 전처리는 **정규 표현식(Regular Expression)**을 사용하여 **구두점(punctuation)**을 제거하여 단어를 **토크나이징(tokenizing)**하는 일에 방해가 되지 않도록 정제하는 것을 목표로 함

In [3]:
# 전처리 함수
def preprocess_sentence(sentence):
    sentence = sentence.lower().strip()

  # 단어와 구두점(punctuation) 사이 거리 만들기
  # 예를 들어서 "I am a student." => "I am a student ."와 같이
  # student와 온점 사이에 거리 만들기
    sentence = re.sub(r"([?.!,])", r" \1 ", sentence)
    sentence = re.sub(r'[" "]+', " ", sentence)

  # (a-z, A-Z, ".", "?", "!", ",")를 제외한 모든 문자를 공백인 ' '로 대체
    sentence = re.sub(r"[^가-힣a-zA-Z0-9?.!,]+", " ", sentence)
    sentence = sentence.strip()
    return sentence
print("슝=3")

슝=3


- 데이터를 로드하는 동시에 전처리 함수를 호출하여 질문과 답변의 쌍을 전처리

In [4]:
# 질문과 답변의 쌍인 데이터셋을 구성하기 위한 데이터 로드 함수
def load_conversations():
    inputs, outputs = [], []
    
    for i in range(len(data) - 1):
        # 전처리 함수를 질문에 해당되는 inputs와 답변에 해당되는 outputs에 적용
        inputs.append(preprocess_sentence(data['Q'].values[i]))
        outputs.append(preprocess_sentence(data['A'].values[i]))
  
    return inputs, outputs
print("슝=3")

슝=3


- 로드한 데이터의 샘플 수 확인

In [5]:
# 데이터를 로드하고 전처리하여 질문을 questions, 답변을 answers에 저장
questions, answers = load_conversations()
print('전체 샘플 수 :', len(questions))
print('전체 샘플 수 :', len(answers))

전체 샘플 수 : 11822
전체 샘플 수 : 11822


- 질문과 답변은 병렬적으로 구성되는 데이터셋이므로 두 샘플 수는 정확하게 일치해야 함
- 둘 다 11,822 개의 샘플이 저장되었음
- 임의로 샘플을 출력해서 질문과 답변이 병렬적으로 잘 저장은 되었는지, 그리고 전처리 함수에서 의도했던 전처리가 진행되었는지 확인해 보기

In [6]:
import random

#랜덤으로 인덱스 뽑아서 5개만 확인할 것.
length = list(range(0,11822,1))
random.shuffle(length)

for i in (length[:5]) :
    print('전처리 후의 {}번째 질문 샘플: {}'.format(i+1, questions[i]))
    print('전처리 후의 {}번째 답변 샘플: {}'.format(i+1, answers[i]))

전처리 후의 8231번째 질문 샘플: 진짜헤어졌네 결국
전처리 후의 8231번째 답변 샘플: 많이 힘들었을 거라 생각해요 .
전처리 후의 10915번째 질문 샘플: 이렇게 또 혼자 좋아하고 이별하고 .
전처리 후의 10915번째 답변 샘플: 짝사랑이 그런가봐요 . 슬프네요 .
전처리 후의 4420번째 질문 샘플: 차 팔아서 불편해
전처리 후의 4420번째 답변 샘플: 불편함을 조금 감수해보세요 .
전처리 후의 8871번째 질문 샘플: 2년 만났어
전처리 후의 8871번째 답변 샘플: 딱 좋을 때네요 .
전처리 후의 2507번째 질문 샘플: 소개받았는데 카톡으로 연락 중
전처리 후의 2507번째 답변 샘플: 썸에서 연인으로 성공하길 바라요 .


## 4. Subword TextEncoder 사용하기

- 한국어 데이터는 형태소 분석기를 사용하여 토크나이징을 해야 한다고 알고 있음
- 하지만, 여기서는 형태소 분석기가 아닌 이 실습에서 사용했던 내부 단어 토크나이저인 'SubwordTextEncoder'를 그대로 사용

- 질문과 답변의 셋을 각각 questions와 answers에 저장하였으므로, 본격적으로 전처리를진행
- 이번 스텝에서 진행할 전체적인 과정을 요약하면 다음과 같음
    - 1. Tensorflow Datasets **SubwordTextEncoder**를 토크나이저로 사용. 단어보다 더 작은 단위인 Subword를 기준으로 토크나이징하고, 각 토큰을 고유한 **정수로 인코딩
    - 2. 각 문장을 토큰화하고 각 문장의 시작과 끝을 나타내는 'START_TOKEN' 및 'END_TOKEN'을 추가
    - 3. 최대 길이 'MAX_LENGTH'인 40을 넘는 문장 필터링
    - 4. MAX_LENGTH보다 길이가 짧ㅇ른 문장들을 40에 맞도록 *패딩*

### (1) 단어장(Vacabulary) 만들기
- 각 단어에 고유한 정수 인덱스를 부여하기 위해서 단어장(Vocabulary) 만들어 보기
- 단어장을 만들 때는 질문과 답변 데이터셋을 모두 사용하여 만듬

In [7]:
import tensorflow_datasets as tfds
print("살짝 오래 걸릴 수 있어요. 스트레칭 한 번 해볼까요? 👐")

# 질문과 답변 데이터셋에 대해서 Vocabulary 생성. (Tensorflow 2.3.0 이상) (클라우드는 2.4)
tokenizer = tfds.deprecated.text.SubwordTextEncoder.build_from_corpus(questions + answers, target_vocab_size=2**13)
print("슝=3 ")

살짝 오래 걸릴 수 있어요. 스트레칭 한 번 해볼까요? 👐
슝=3 


- 이때, 디코더의 문장 생성 과정에서 사용할 '시작 토큰'과 '종료 토큰'에 대해서도 임의로 단어장에 추가하여서 정수를 부여
- 이미 생성된 번화와 겹치지 않도록 각각 단어장의 크기와 그보다 1이 큰 수를 번호로 부여하면 됨

In [8]:
# 시작 토큰과 종료 토큰에 고유한 정수 부여
START_TOKEN, END_TOKEN = [tokenizer.vocab_size], [tokenizer.vocab_size + 1]
print("슝=3")

슝=3


- 시작 토큰과 종료 토큰에 부여된 정수를 출력

In [9]:
print('START_TOKEN의 번호 :' ,[tokenizer.vocab_size])
print('END_TOKEN의 번호 :' ,[tokenizer.vocab_size + 1])

START_TOKEN의 번호 : [8161]
END_TOKEN의 번호 : [8162]


- 각각 8331, 8332라는 점에서 현재 단어장의 크기가 8,331(0번부터 8,330번)이라는 의미
- 2개의 토큰을 추가해 주었기 때문에 단어장의 크기도 +2임을 명시

In [ ]:
# 시작 토큰과 종료 토큰을 고려하여 +2를 하여 단어장의 크기 산정
VOCAB_SIZE = tokenizer.vocab_size + 2
print(VOCAB_SIZE)

## 5. 모델 구성하기

- 트랜스포머 모델을 구현

## 6. 모델 평가하기

- 전처리 방법을 고려하여 입력된 문장에 대해서 대답을 얻는 예측 함수 만들기

- 예측(inference) 단계는 기본적으로 다음과 같은 과정을 거침
    - 1. 새로운 입력 문장에 대해서는 훈련 때와 동일한 전처리 거침
    - 2. 입력 문장을 토크나이징하고, 'START_TOKEN'과 'END_TOKEN'ㅇ르 추가
    - 3. 패딩 마스킹과 룩 어헤드 마스킹을 계산
    - 4. 디코더는 입력 시퀀스로부터 다음 단어를 예측
    - 5. 디코더는 예측한 다음 단어를 기존의 입력 시퀀스에 추가하여 새로운 입력으로 사용
    - 6. 'END_TOKEN'이 예측되거나 문장의 최대 길이에 도달하면 디코더는 동작을 멈춤
- 위의 과정을 모두 담은 'decoder_inference()' 함수 만들기

In [ ]:
def decoder_inference(sentence):
    sentence = preprocess_sentence(sentence)
    
    # 입력된 문장을 정수 인코딩 후, 시작 토큰과 종료 토큰을 앞뒤로 추가
    # ex) Where have you been? → [[8331   86   30    5 1059    7 8332]]
    sentence = tf.expand_dims(
        START_TOKEN + tokenizer.encode(sentence) + END_TOKEN, axis=0)
    
    # 디코더의 현재까지의 예측한 출력 시퀀스가 지속적으로 저장되는 변수
    # 처음에는 예측한 내용이 없음으로 시작 토큰만 별도 저장. ex) 8331
    output_sequence = tf.expand_dims(START_TOKEN, 0)
    
    # 디코더의 인퍼런스 단계
    for i in range(MAX_LENGTH):
        # 디코더는 최대 MAX_LENGTH의 길이만큼 다음 단어 예측을 반복
        predictions = model(inputs=[sentence, output_sequence], training=False)
        predictions = predictions[:, -1:, :]
        
        # 현재 예측한 단어의 정수
        predicted_id = tf.cast(tf.argmax(predictions, axis=-1), tf.int32)
        
        # 만약 현재 예측한 단어가 종료 토큰이라면 for문을 종료
        if tf.equal(predicted_id, END_TOKEN[0]):
            break
            
        # 예측한 단어들은 지속적으로 output_sequence에 추가됨
        # 이 output_sequence는 다시 디코더의 입력이 됨
        output_sequence = tf.concat([output_sequence, predicted_id], axis=-1)

    return tf.squeeze(output_sequence, axis=0)
print("슝=3")

- 임의의 입력 문장에 대해서 'decoder_inference()' 함수를 호출하여 챗봇의 대답을 얻는 'sentence_generation()' 함수 만들기

In [ ]:
def sentence_generation(sentence):
  # 입력 문장에 대해서 디코더를 동작 시켜 예측된 정수 시퀀스를 리턴받음
    prediction = decoder_inference(sentence)
    
    # 정수 시퀀스를 다시 텍스트 시퀀스로 변환
    predicted_sentence = tokenizer.decode(
      [i for i in prediction if i < tokenizer.vocab_size])
    
    print('입력 : {}'.format(sentence))
    print('출력 : {}'.format(predicted_sentence))

    return predicted_sentence
print("슝=3")

- 임의의 문장으로부터 챗봇의 대답 얻어보기

In [ ]:
sentencee_generation('나 지금 배고파')

In [ ]:
sentencee_generation('나 지금 피곤해')